In [1]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain_openai import ChatOpenAI

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
Key = os.getenv("OPENAI_API_KEY")

In [22]:
llm = ChatOpenAI(openai_api_key= Key, model_name = "gpt-3.5-turbo", temperatur = 0.5)

d:\Projects\mcqGenerator\mcq\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! temperatur is not default parameter.
                temperatur was transferred to model_kwargs.
                Please confirm that temperatur is what you intended.
  if await self.run_code(code, result, async_=asy):
